In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Recognize all aspect terms with their corresponding sentiment polarity in thegiven below column. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the cloth. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\"."
               ```{text}```
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)



In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/Womens Clothing E-Commerce Reviews.csv'
df = pd.read_csv(file_path)


In [ ]:

  >>> import nltk
  >>> nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.corpus import stopwords

# Drop rows with missing 'Review Text'
df.dropna(subset=['Review Text'], inplace=True)

# Convert text to lowercase
df['Cleaned_Review_Text'] = df['Review Text'].str.lower()

# Remove special characters and numbers using regex
df['Cleaned_Review_Text'] = df['Cleaned_Review_Text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

# Optional: Remove stopwords
stop_words = set(stopwords.words('english'))
df['Cleaned_Review_Text'] = df['Cleaned_Review_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Show the first few rows after cleaning
df[['Review Text', 'Cleaned_Review_Text']].head()

,Review Text,Cleaned_Review_Text
0,Absolutely wonderful - silky and sexy and comf...,absolutely wonderful silky sexy comfortable
1,Love this dress! it's sooo pretty. i happene...,love dress sooo pretty happened find store im ...
2,I had such high hopes for this dress and reall...,high hopes dress really wanted work initially ...
3,"I love, love, love this jumpsuit. it's fun, fl...",love love love jumpsuit fun flirty fabulous ev...
4,This shirt is very flattering to all due to th...,shirt flattering due adjustable front tie perf...


In [ ]:
# Continue with data preprocessing without removing stopwords

# Convert text to lowercase
df['Cleaned_Review_Text'] = df['Review Text'].str.lower()

# Remove special characters and numbers using regex
df['Cleaned_Review_Text'] = df['Cleaned_Review_Text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

# Show the first few rows after cleaning
df[['Review Text', 'Cleaned_Review_Text']].head()
print(df['Cleaned_Review_Text'][0])

absolutely wonderful  silky and sexy and comfortable


In [ ]:
print(type(df['Cleaned_Review_Text']))


<class 'pandas.core.series.Series'>


In [ ]:
import pandas as pd
from langchain import PromptTemplate, LLMChain

# Define the prompt template
template = """
    Recognize all aspect terms with their corresponding sentiment polarity in the given below column. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the cloth. Determine the sentiment polarity from the options ["positive", "negative", "neutral"]. Answer in the format ["aspect", "sentiment"] without any explanation. If no aspect term exists, then only answer "[]".
    ```{text}```
"""

# Create the PromptTemplate object
prompt = PromptTemplate(template=template, input_variables=["text"])

# Initialize LLMChain with the prompt and your LLM (which needs to be defined)
llm_chain = LLMChain(prompt=prompt, llm=llm)  # Replace 'your_llm_here' with your actual LLM

# Read the DataFrame (assuming df is already loaded with 'Cleaned_Review_Text' column)
# df = pd.read_csv('your_dataframe.csv')

# Initialize an empty list to store results
results = []

# Loop through the 'Cleaned_Review_Text' column
# for review in df['Cleaned_Review_Text']:
df.reset_index(drop=True, inplace=True)
for review in range(len(df['Cleaned_Review_Text'])):
    response = llm_chain.generate({'text': df['Cleaned_Review_Text'][review]})
    results.append(response)

# for review in df['Cleaned_Review_Text']:
#     response = llm_chain.generate({'text': review})
#     results.append(response)

# Save the results to a new CSV file
result_df = pd.DataFrame(results, columns=['Aspect', 'Sentiment'])
result_df.to_csv('aspect_sentiment_results.csv', index=False)


KeyError: ignored

In [ ]:
import torch

# Your code that uses GPU here

# Free unused GPU memory
torch.cuda.empty_cache()

In [ ]:
import pandas as pd
from langchain import PromptTemplate, LLMChain

# Define the prompt template
template = """
    Recognize all aspect terms with their corresponding sentiment polarity in the given below column.
    The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the cloth.
    Determine the sentiment polarity from the options ["positive", "negative", "neutral"]. Answer in the format ["aspect", "sentiment"] without any explanation.
    If no aspect term exists, then only answer "[]".
    {text}
"""

# Create the PromptTemplate object
prompt = PromptTemplate(template=template, input_variables=["text"])

# Initialize LLMChain with the prompt and your LLM (which needs to be defined)
llm_chain = LLMChain(prompt=prompt, llm=llm)  # Replace 'your_llm_here' with your actual LLM

# Read the DataFrame (assuming df is already loaded with 'Cleaned_Review_Text' column)
# df = pd.read_csv('your_dataframe.csv')

# Initialize an empty list to store results
results = []

# Loop through the 'Cleaned_Review_Text' column
# for review in df['Cleaned_Review_Text']:
df.reset_index(drop=True, inplace=True)
for index, row in df.iterrows():
    # print(row['Cleaned_Review_Text'])
    response = llm_chain(row['Cleaned_Review_Text'])
    results.append(response)
# for review in df['Cleaned_Review_Text']:
#     response = llm_chain.generate({'text': review})
#     results.append(response)

# Save the results to a new CSV file
result_df = pd.DataFrame(results, columns=['Aspect', 'Sentiment'])
# result_df.to_csv('aspect_sentiment_results.csv', index=False)
result_df.to_csv('aspect_sentiment_results.csv', index=False)


ModuleNotFoundError: ignored

In [ ]:
import torch
torch.cuda.empty_cache()